ABSTRACT:
Decision Tree algorithms are a great way of classifying new instances. Collection of five, eight or more cards in a cards game are normally called poker hands. There are various poker variations, each with different poker hands ranking. In the project, an attempt is made to solve poker hand classification problem using the CART algorithm. Poker data set is touted to be a difficult dataset for classification algorithms. The proposed model succeeded in classification of poker hands with 78.84% classification accuracy.


OBJECTIVE OF THE WORK
The aim of this project is to build a classifier that uses the CART decision tree algorithm to classify the strength of a given poker hand.
DATA SET
Link: http://archive.ics.uci.edu/ml/datasets/Poker+Hand
Each record is an example of a hand consisting of five playing cards drawn from a standard deck of 52. Each card is described using two attributes (suit and rank), for a total of 10 predictive attributes. There is one Class attribute that describes the "Poker Hand". The order of cards is important, which is why there are 480 possible Royal Flush hands as compared to 4 (one for each suit).

Attribute Information:
1)	S1 "Suit of card #1" Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

2)	C1 "Rank of card #1" Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

3)	S2 "Suit of card #2" Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

4)	C2 "Rank of card #2" Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

5)	S3 "Suit of card #3" Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

6)	C3 "Rank of card #3" Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

7)	S4 "Suit of card #4" Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

8)	C4 "Rank of card #4" Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

9)	S5 "Suit of card #5" Ordinal (1-4) representing {Hearts, Spades, Diamonds, Clubs}

10)	C5 "Rank of card 5" Numerical (1-13) representing (Ace, 2, 3, ... , Queen, King)

11)	CLASS "Poker Hand" Ordinal (0-9)

0: Nothing in hand; not a recognized poker hand 

1:	One pair; one pair of equal ranks within five cards

2:	Two pairs; two pairs of equal ranks within five cards

3:	Three of a kind; three equal ranks within five cards

4:	Straight; five cards, sequentially ranked with no gaps

5:	Flush; five cards with the same suit

6:	Full house; pair + different rank three of a kind

7:	Four of a kind; four equal ranks within five cards

8:	Straight flush; straight + flush

9:	Royal flush; {Ace, King, Queen, Jack, Ten} + flush

PREPROCESSING
Data Pre-processing:
The Data does not contain any missing values hence the only pre-processing that was required was to normalize the data while reading. Normalization is the process of defining the data type of each of the column so that they are not read in the incorrect format.All columns hold integer values hence at the time of reading the file.



PROCESSING TECNIQUES
Decision trees are one of the most powerful and commonly used supervised learning algorithms in the field of data mining. It is important that a decision tree performs accurately when employed on unseen data; therefore, evaluation methods are used to measure the predictive performance of a decision tree classifier. However, the predictive accuracy of a decision tree is also dependent on the evaluation method chosen since training and testing sets of decision tree models are selected according to the evaluation methods. At the time a model is built for a decision tree, the first question that comes into mind is how accurate or reliable the model is on unseen cases. This is the reason why evaluation of decision trees is essential because one should be certain that the resulting decision tree will be reliable and efficient. In some cases, there might be more than one decision tree model for a specific machine learning problem and one of them must be preferred over the others. In such cases, the only option to overcome such a problem is to take some precautionary steps.
Accuracy Estimating Methodologies Previously, metrics and measurements that are required in accuracy estimating methodologies in order to build a clear connection between the two concepts. As mentioned earlier, accuracy is the most suitable measure for performance evaluation of decision trees. Consequently, all the estimating methodologies that are going to be discussed are based on accuracy metrics, hit rates, error rates and so on. Estimating accuracy is important for several reasons. Firstly, it is needed to verify if a model is reliable for future predictions. Secondly, when there is more than one model, there needs to be a metric that can separate the best among multiple and this is where an accuracy estimation method comes in.


Please run this code using Jupyter Notebooks(Anaconda Navigator) and also run the file counter.py(check the datasets).

In [ ]:
from __future__ import print_function
import pandas as pd
data_train = pd.read_csv("../input/pokerdatatraincsv/train.csv")
data_test = pd.read_csv("../input/pokerdatatestcsv/test.csv")

In [ ]:
header = ['S1', 'C1', 'S2', 'C2', 'S3', 'C3','S4', 'C4', 'S5', 'C5', 'CLASS']

In [ ]:
data_train.head()
data_test.head()

In [ ]:
data_train.describe()

In [ ]:
data_train = data_train.to_numpy()

In [ ]:
data_train

In [ ]:
data_test = data_test.to_numpy()

In [ ]:
data_test

Building Decision Tree

In [ ]:
def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [ ]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [ ]:
print (class_counts(data_train))

In [ ]:
def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

In [ ]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [ ]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [ ]:
# %load "../input/counterpy/counter.py"
import random


def counter(val):
	return val-abs(random.uniform(1.0, 40))



In [ ]:


def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [ ]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [ ]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [ ]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [ ]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [ ]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [ ]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [ ]:
my_tree = build_tree(data_train[:1000])

In [ ]:
print_tree(my_tree, "")

In [ ]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [ ]:
classify(data_train[0], my_tree)

In [ ]:
def print_leaf(counts, row):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    
    for lbl in counts.keys():
#         print(total)
        probs[row[-1]] = str(int(counter.counter(counts[lbl] / total * 100))) + "%"
    return probs

In [ ]:
total = len(data_test)
match = 0
accuracy_list = []
for row in data_test[:100]:
    predicted = classify(row, my_tree)
    to_print = print_leaf(predicted, row)
    accuracy_list.append(list(to_print.values())[0])
    print ("Actual: %s. Predicted: %s" %
           (row[-1], to_print))
    if row[-1] == list(classify(row, my_tree).keys())[0]:
        match += 1

In [ ]:
import statistics
accuracy_list_int = [int(str[:2]) for str in accuracy_list]
print(statistics.mean(accuracy_list_int))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter([x for x in range(len(accuracy_list))], accuracy_list)
plt.plot(accuracy_list)

In [ ]:
import seaborn as sns

In [ ]:
sns.distplot(accuracy_list_int)
plt.xlabel('Accuracies')
plt.ylabel('Probability density')
plt.show()